In [24]:
import numpy as np
import pandas as pd

# Cálculo Numérico - EP 2

## Funções

In [119]:
def root_by_GE(matrix):
    n = len(matrix)
    matrix = matrix.astype(float)

    for i in range(n):
        max_row = i + np.argmax(abs(matrix[i:, i]))
        if abs(matrix[max_row][i]) < 1e-12:
            raise ValueError('Matriz singular, não há solução única.')
        if i != max_row:
            matrix[[i, max_row]] = matrix[[max_row, i]]
            print(f'Pivotamento: Trocando linha {i+1} com linha {max_row+1}')
            print(matrix, '\n')
        pivot = matrix[i, i]
        matrix[i] /= pivot
        print(f'Normalizando linha {i+1} por {pivot}')
        print(matrix, '\n')
        for k in range(i + 1, n):
            factor = matrix[k, i]
            matrix[k] -= factor * matrix[i]
            print(f'Eliminando elemento abaixo do pivô na linha {k+1}')
            print(matrix, '\n')

    for i in range(n-1, -1, -1):
        for k in range(i - 1, -1, -1):
            factor = matrix[k, i]
            matrix[k] -= factor * matrix[i]
            print(f'Eliminando elemento acima do pivô na linha {k+1}')
            print(matrix, '\n')

    solution = matrix[:, -1]
    return matrix, solution

def root_by_jacobi(matrix, b, x0, error_acceptance, iterations=10, save_data=False, data_name='jacobi_data'):
    x = np.zeros(len(matrix))
    data = {'k': [], 'x': [], 'error': []}
    for it in range(iterations):
        x0 = np.copy(x)
        for i in range(len(x)):
            prod = 0
            for j in range(len(x)):
                if j != i:
                    prod += matrix[i][j] * x0[j]
            x[i] = (b[i] - prod) / matrix[i][i]

        epsilon = np.max(np.abs(x-x0))

        data['k'].append(it+1)
        data['x'].append(np.round(np.copy(x), 4))
        data['error'].append(round(epsilon, 4))
        
        if epsilon < error_acceptance:
            break

    df = pd.DataFrame(data=data)
    if save_data:
        df.to_csv(f'{data_name}.csv', index=False)
    
    return x, df

def root_by_GS(matrix, b, x, error_acceptance, iterations=10, save_data=False, data_name='gauss_seidel_data'):
    size = len(matrix)
    beta = np.ones(size)
    new_matrix = np.zeros((size, size))
    data = {'k': [], 'x': [], 'error': []}

    for i in range(size):
        max_location = np.argmax(np.abs(matrix[i]))
        new_matrix[i] = matrix[max_location]

    for i in range(size):
        for j in range(size):
            if j != i:
                beta[i] += beta[j] * np.abs(new_matrix[i][j]) / np.abs(new_matrix[i][i])
        beta[i] -= 1

    if np.max(beta) >= 1:
        raise ValueError('Matriz não satisfaz o critério de Sassenfeld.')

    for it in range(iterations):
        x_old = np.copy(x)
        for i in range(size):
            prod = 0
            for j in range(size):
                if j != i:
                    prod += matrix[i][j] * x[j]
            x[i] = (b[i] - prod) / matrix[i][i]
        
        epsilon = np.max(np.abs(x - x_old))
        
        data['k'].append(it+1)
        data['x'].append(np.round(np.copy(x), 4))
        data['error'].append(round(epsilon, 4))

        if epsilon < error_acceptance:
            break

    df = pd.DataFrame(data=data)
    if save_data:
        df.to_csv(f'{data_name}.csv', index=False)

    return x, df

## Item b)

In [47]:
matrix = np.array([[0, 5.3, -1.8, 3.1], [11.9, 0, 1.8, 15], [1, -1, -1, 0]])
final_matrix, solution = root_by_GE(matrix)
print('Solução:', solution)

Pivotamento: Trocando linha 1 com linha 2
[[11.9  0.   1.8 15. ]
 [ 0.   5.3 -1.8  3.1]
 [ 1.  -1.  -1.   0. ]] 

Normalizando linha 1 por 11.9
[[ 1.         0.         0.1512605  1.2605042]
 [ 0.         5.3       -1.8        3.1      ]
 [ 1.        -1.        -1.         0.       ]] 

Eliminando elemento abaixo do pivô na linha 2
[[ 1.         0.         0.1512605  1.2605042]
 [ 0.         5.3       -1.8        3.1      ]
 [ 1.        -1.        -1.         0.       ]] 

Eliminando elemento abaixo do pivô na linha 3
[[ 1.         0.         0.1512605  1.2605042]
 [ 0.         5.3       -1.8        3.1      ]
 [ 0.        -1.        -1.1512605 -1.2605042]] 

Normalizando linha 2 por 5.3
[[ 1.          0.          0.1512605   1.2605042 ]
 [ 0.          1.         -0.33962264  0.58490566]
 [ 0.         -1.         -1.1512605  -1.2605042 ]] 

Eliminando elemento abaixo do pivô na linha 3
[[ 1.          0.          0.1512605   1.2605042 ]
 [ 0.          1.         -0.33962264  0.58490566]

## Item c)

In [120]:
jacobi_matrix = np.array([[11.9, 0.0, 1.8], [0.0, 5.3, -1.8], [1, -1, -1]])
jacobi_b = np.array([15.0, 3.1, 0.0])
jacobi_x0 = np.array([1, 1, 1])

jacobi_solution, jacobi_data = root_by_jacobi(jacobi_matrix, jacobi_b, jacobi_x0, 1e-3, iterations=50, save_data=True)
print('Solução:', jacobi_solution)
jacobi_data

Solução: [1.19184657 0.73906147 0.45390323]


,k,x,error
0,1,"[1.2605, 0.5849, -0.0]",1.2605
1,2,"[1.2605, 0.5849, 0.6756]",0.6756
2,3,"[1.1583, 0.8144, 0.6756]",0.2294
3,4,"[1.1583, 0.8144, 0.344]",0.3316
4,5,"[1.2085, 0.7017, 0.344]",0.1126
5,6,"[1.2085, 0.7017, 0.5068]",0.1628
6,7,"[1.1839, 0.757, 0.5068]",0.0553
7,8,"[1.1839, 0.757, 0.4268]",0.0799
8,9,"[1.1959, 0.7299, 0.4268]",0.0271
9,10,"[1.1959, 0.7299, 0.4661]",0.0392


## Item e)

In [122]:
gs_matrix = np.array([[11.9, 0.0, 1.8], [0.0, 5.3, -1.8], [1, -1, -1]])
gs_b = np.array([15.0, 3.1, 0.0])
gs_x0 = np.array([1, 1, 0.5])

gs_solution, gs_data = root_by_GS(gs_matrix, gs_b, gs_x0, 1e-3, iterations=50, save_data=True)
print('Solução:', gs_solution)
gs_data

Solução: [1.19186087 0.73902937 0.45283149]


,k,x,error
0,1,"[1.1849, 0.7547, 0.4302]",0.2453
1,2,"[1.1954, 0.731, 0.4644]",0.0343
2,3,"[1.1903, 0.7426, 0.4476]",0.0168
3,4,"[1.1928, 0.7369, 0.4559]",0.0083
4,5,"[1.1915, 0.7397, 0.4518]",0.0041
5,6,"[1.1922, 0.7384, 0.4538]",0.0020
6,7,"[1.1919, 0.739, 0.4528]",0.0010
